In [1]:
# imports
import re
import tensorflow as tf
import numpy as np
import os
import time
import datetime
import data_helpers
from collections import defaultdict
import pytreebank
import gensim
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Input, GlobalMaxPooling1D, Conv1D, MaxPooling1D, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.initializers import Constant
import tensorflow.keras.backend

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize

from sklearn.model_selection import train_test_split

C:\Users\mystery\AppData\Local\Programs\Python\Python36\lib\site-packages\gensim\utils.py:1212: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
# use nltk to remove stopwords and lemmatize
# you might need to run: nltk.download() to fetch the stopword package in "all packages"
# you might also need to run ntlk.download("punkt")
english_stopwords = set(stopwords.words("english"))
wordnet_lemmatizer = WordNetLemmatizer()

In [3]:
def clean_input_text(text):
    clean_text = []
    for sent in text:
        clean_sent = ""
        sent_tokens = word_tokenize(sent)
        for token in sent_tokens:
            clean_sent += wordnet_lemmatizer.lemmatize(token) + " " if token not in english_stopwords else ""
        clean_text.append(clean_sent)
    return clean_text

In [4]:
# hyperparameters for the cnn
MAX_SEQUENCE_LENGTH = 1000
MAX_NUM_WORDS = 20000
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.2
TEST_SPLIT = 0.2
NUMBER_DIFFERENT_OUTPUTS = 5

In [5]:
# load pretrain glove word2vec instance for preprocessing
filename = './data/glove.6B.300d.txt'

print('Indexing word vectors.')

embeddings_index = {}
with open(filename, encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Indexing word vectors.
Found 400000 word vectors.


In [6]:
# load SST1
dataset = pytreebank.load_sst("./data/stanfordSentimentTreebank")

example = dataset["dev"]
len(example)


1101

In [7]:
dataset["train"][1].to_labeled_lines()

# retrieve all sentences and their labels from the sst dataset
def retrieve_examples_and_labels(dataset_name):
    labels, x_text = [],[]
    for lines in dataset[dataset_name]:
        labeled_lines = lines.to_labeled_lines()
        for label, example in labeled_lines:
            labels.append(label)
            x_text.append(example)
    return x_text, labels

x_text1, y_train1 = retrieve_examples_and_labels("train")
x_text2, y_train2 = retrieve_examples_and_labels("test")
x_text = clean_input_text(x_text1 + x_text2)
y = y_train1 + y_train2



In [8]:
# vectorize the input text (both negative and positive )
tokenizer = Tokenizer(num_words=MAX_NUM_WORDS)
tokenizer.fit_on_texts(x_text)
sequences = tokenizer.texts_to_sequences(x_text)
word_index = tokenizer.word_index
print(len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
labels[0]


15844
Shape of data tensor: (401182, 1000)
Shape of label tensor: (401182, 5)


array([0., 0., 0., 1., 0.], dtype=float32)

In [9]:
# prepare embedding matrix
num_words = min(MAX_NUM_WORDS, len(word_index)) + 1
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))
for word, i in word_index.items():
    if i > MAX_NUM_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [10]:
# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]
len(y_val)

80236

In [11]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [12]:
# train a 1D convnet with global maxpooling
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = GlobalMaxPooling1D()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(NUMBER_DIFFERENT_OUTPUTS, activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

In [13]:
# create a datagenerator inheriting from utils.Sequence
# in order to feed data by batches to the cnn to avoid
# gpu problems.
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding (Embedding)        (None, 1000, 300)         4753500   
_________________________________________________________________
conv1d (Conv1D)              (None, 996, 128)          192128    
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 199, 128)          0         
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 195, 128)          82048     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 39, 128)           0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 35, 128)           82048     
__________

In [14]:
model.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          validation_data=(x_val, y_val))

Train on 320946 samples, validate on 80236 samples
Epoch 1/10
320946/320946 [==============================] - 173s 538us/step - loss: 0.7874 - acc: 0.7306 - val_loss: 0.7630 - val_acc: 0.7411
Epoch 2/10
320946/320946 [==============================] - 168s 525us/step - loss: 0.7387 - acc: 0.7489 - val_loss: 0.7482 - val_acc: 0.7449
Epoch 3/10
320946/320946 [==============================] - 178s 553us/step - loss: 0.7157 - acc: 0.7590 - val_loss: 0.7376 - val_acc: 0.75034s - los - ETA: 2s - loss: 0.7156 - ac
Epoch 4/10
320946/320946 [==============================] - 173s 538us/step - loss: 0.7007 - acc: 0.7662 - val_loss: 0.7593 - val_acc: 0.7499
Epoch 5/10
320946/320946 [==============================] - 168s 524us/step - loss: 0.6918 - acc: 0.7707 - val_loss: 0.7754 - val_acc: 0.7511 -
Epoch 6/10
320946/320946 [==============================] - 169s 527us/step - loss: 0.6859 - acc: 0.7743 - val_loss: 0.7676 - val_acc: 0.7513
Epoch 7/10
320946/320946 [==============================]